In [1]:
%jsroot on

In [2]:
#include<TFile.h>
#include<TH1.h>
#include<TMath.h>
#include<TF1.h>
#include<TLine.h>
#include<TPoint.h>
#include<iostream>
#include<string>
#include<stdio.h>

In [3]:
TCanvas *c1 = new TCanvas("c1","The FillRandom example",200,10,700,900);

auto pad3 = new TPad("pad3","h_data",0.05,0.68,0.95,0.96);
auto pad1 = new TPad("pad1","The pad with the function",0.05,0.36,0.95,0.64);
auto pad2 = new TPad("pad2","The pad with the histogram",0.05,0.04,0.95,0.32);

pad1->Draw();
pad2->Draw();
pad3->Draw();

In [4]:
pad3->cd();
const Int_t n = 20;

TH1D *h_data = new TH1D("h_data", "h_data", n, 110, 140); // Histograma de datos.
h_data->SetBinContent(1, 0);
h_data->SetBinContent(2, 1);
h_data->SetBinContent(3, 0);
h_data->SetBinContent(4, 1);
h_data->SetBinContent(5, 1);
h_data->SetBinContent(6, 6);
h_data->SetBinContent(7, 7);
h_data->SetBinContent(8, 2);
h_data->SetBinContent(9, 22);
h_data->SetBinContent(10, 30);
h_data->SetBinContent(11, 27);
h_data->SetBinContent(12, 20);
h_data->SetBinContent(13, 13);
h_data->SetBinContent(14, 9);
h_data->SetBinContent(15, 9);
h_data->SetBinContent(16, 13);
h_data->SetBinContent(17, 19);
h_data->SetBinContent(18, 11);
h_data->SetBinContent(19, 9);
h_data->SetBinContent(20, 0);

h_data->SetBinError(1, 0.38974303 );
h_data->SetBinError(2, 0.536510944);
h_data->SetBinError(3, 0.529702604);
h_data->SetBinError(4, 0.642001867);
h_data->SetBinError(5, 0.969341516);
h_data->SetBinError(6, 1.47611344);
h_data->SetBinError(7, 1.69797957);
h_data->SetBinError(8, 3.28577447);
h_data->SetBinError(9, 5.40784931);
h_data->SetBinError(10, 9.10106468);
h_data->SetBinError(11, 9.73541737);
h_data->SetBinError(12, 5.55019951);
h_data->SetBinError(13, 3.57914758);
h_data->SetBinError(14, 2.77877331);
h_data->SetBinError(15, 3.23697519);
h_data->SetBinError(16, 4.3608489);
h_data->SetBinError(17, 5.77172089);
h_data->SetBinError(18, 3.38666105);
h_data->SetBinError(19, 2.98861837);
h_data->SetBinError(20, 1.58402085);
h_data->Draw("E1");
pad3->SetGridx();
pad3->SetGridy();

//auto pdf = new TF1("pdf","[0]*(2*sqrt(2)*[2]*[1]*(sqrt(pow([2],2)*(pow([2],2)+pow([1],2))))/(TMath::Pi()*sqrt(pow([2],2)+(sqrt(pow([2],2)*(pow([2],2)+pow([1],2)))))))/((pow((pow(x,2)-pow([2],2)),2))+(pow([2],2)*pow([1],2)))",110,140); //Función de Densidad de Probabilidad.
auto pdf = new TF1("pdf","expo",110,140);
//pdf->SetParameters(1,h_data->GetStdDev(),h_data->GetMean());
//pdf->SetParNames ("Constant","Standard deviation","Mean");
h_data->Fit("pdf","R"); // Puede usarse V para agregar más detalles estadísticos.
pdf->SetTitle("Fit");

c1->Update();

 FCN=112.636 FROM MIGRAD    STATUS=CONVERGED      99 CALLS         100 TOTAL
                     EDM=1.15685e-07    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant    -7.69500e+00   1.57691e+00   7.58703e-04   4.96689e-04
   2  Slope        6.83824e-02   1.23309e-02   5.93413e-06   2.46817e-02


In [5]:
pad2->cd();
TH1D *h_data_f = new TH1D("h_data_f","Test random numbers",n,110,140);

//h_data_f->SetTitle("Weighted Histogram");
//h_data_f->SetFillColor(45);
h_data_f->SetLineColor(kGreen);

for (int i=0; i<n+1; i++) h_data_f->SetBinContent(i,pdf->Eval(h_data->GetXaxis()->GetBinCenter(i)));

h_data_f->Draw("E1 same");

c1->Update();

In [6]:
pad1->cd();
pad1->SetGridx();
pad1->SetGridy();
pad1->GetFrame()->SetBorderMode(-1);
pad1->GetFrame()->SetBorderSize(5);
pdf->SetLineColor(4);
pdf->SetLineWidth(6);
pdf->Draw();
c1->Update();

In [7]:
// TFile myfile("fillrandom.root","RECREATE");
// pdf->Write();
// h_data_f->Write();

In [8]:
gROOT->GetListOfCanvases()->Draw()

In [9]:
//apply the chi2 test and retrieve the residuals
Double_t res[n], x[n];
h_data->Chi2Test(h_data_f,"UU P",res);

Chi2 = 47.028525, Prob = 0.000209645, NDF = 18, igood = 3


Warning in <TH1D::Chi2TestX>: Both histograms are not unweighted and option UU has been requested
Info in <TH1D::Chi2TestX>: There is a bin in h1 with less than 1 event.

Info in <TH1D::Chi2TestX>: There is a bin in h2 with less than 1 event.



In [10]:
//Graph for Residuals
for (Int_t i=0; i<n; i++) x[i]= 4.+i*12./20.+12./40.;
TGraph *resgr = new TGraph(n,x,res);
resgr->GetXaxis()->SetRangeUser(4,16);
resgr->GetYaxis()->SetRangeUser(-3.5,3.5);
resgr->GetYaxis()->SetTitle("Normalized Residuals");
resgr->SetMarkerStyle(21);
resgr->SetMarkerColor(2);
resgr->SetMarkerSize(.9);
resgr->SetTitle("Normalized Residuals");
 
//Quantile-Quantile plot
TF1 *f = new TF1("f","TMath::Gaus(x,0,1)",-10,10);
TGraphQQ *qqplot = new TGraphQQ(n,res,f);
qqplot->SetMarkerStyle(20);
qqplot->SetMarkerColor(2);
qqplot->SetMarkerSize(.9);
qqplot->SetTitle("Q-Q plot of Normalized Residuals");
 
//create Canvas
TCanvas *c2 = new TCanvas("c2","Chistat Plot",10,10,700,600);
c2->Divide(2,2);
 
// Draw Histogramms and Graphs
c2->cd(1);
h_data->SetMarkerColor(4);
h_data->SetMarkerStyle(20);
 
h_data->Draw("E");
 
c2->cd(2);
h_data_f->Draw("");
h_data_f->SetMarkerColor(4);
h_data_f->SetMarkerStyle(20);
 
c2->cd(3);
gPad->SetGridy();
resgr->Draw("APL");
 
c2->cd(4);
qqplot->Draw("AP");
 
c2->cd(0);

c2->Draw();
c2->Update();

Esqueleto del código:
https://root.cern/doc/v610/fillrandom_8C.html

Ejemplo aplicación Chi2:
https://root.cern/doc/v610/chi2test_8C_source.html
http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.173.6388&rep=rep1&type=pdf

Funciones pruebas de bondad:
KolmogorovTest
Chi2Test